# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234766571483                   -0.50    7.0
  2   -7.250272562748       -1.81       -1.40    1.0
  3   -7.251188257533       -3.04       -2.24    1.0
  4   -7.251311556095       -3.91       -2.32    2.0
  5   -7.251334363066       -4.64       -2.71    1.0
  6   -7.251336358837       -5.70       -2.94    2.0
  7   -7.251338682414       -5.63       -3.61    1.0
  8   -7.251338778732       -7.02       -3.85    2.0
  9   -7.251338794754       -7.80       -4.23    1.0
 10   -7.251338798061       -8.48       -4.75    1.0
 11   -7.251338798567       -9.30       -4.95    3.0
 12   -7.251338798673       -9.98       -5.33    1.0
 13   -7.251338798703      -10.52       -6.05    2.0
 14   -7.251338798704      -11.95       -6.13    3.0
 15   -7.251338798704      -12.50       -6.52    1.0
 16   -7.251338798705      -13.01       -6.80    2.0
 17   -7.251338798705      -13.65       -7.18 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.049470698710326555
[ Info: Arnoldi iteration step 2: normres = 0.4097800955632776
[ Info: Arnoldi iteration step 3: normres = 0.5011641898040461
[ Info: Arnoldi iteration step 4: normres = 0.799229952124047
[ Info: Arnoldi iteration step 5: normres = 0.3325998648798636
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.28e-01, 4.60e-02, 2.20e-01, 2.05e-01, 3.95e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4559410328035674
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.16e-02, 1.71e-01, 4.00e-01, 9.50e-02, 4.35e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08362857680788917
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.35e-03, 2.92e-02, 1.02e-02, 3.19e-02, 6.25e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09734723936036886
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.67e-0